# Import Dependences

In [1]:
import pandas as pd
import numpy as np
import time

# Load the data into dataframes

In [2]:
movies_df = pd.read_csv("../Resources/IMDB_OMDB_processed.csv")
directors_df = pd.read_csv("../Resources/IMDB_OMDB_director_data.csv", encoding='utf-8')
writers_df = pd.read_csv("../Resources/IMDB_OMDB_writer_data.csv", encoding='utf-8') 
actors_df = pd.read_csv("../Resources/IMDB_OMDB_actor_data.csv", encoding='utf-8') 
movies_df.head(3)

,Unnamed: 0,tconst,averageRating,numVotes,isAdult,startYear,runtimeMinutes,genres,Movie,OMDB_Title,...,OMDB_Actors,OMDB_Language,OMDB_Country,OMDB_Metascore,OMDB_imdbRating,OMDB_imdbVotes,OMDB_BoxOffice,OMDB_Production,ReleaseYear,ReleaseMonth
0,0,tt10048376,4.8,30,0,2019,101.0,Drama,Ya podaryu tebe pobedu,Ya podaryu tebe pobedu,...,"Tigran Agadzhanyan, Maria Atlas, Valeriy Barinov",Russian,Russia,NaN,5.1,22.0,NaN,NaN,2019.0,4.0
1,1,tt10048556,3.7,340,0,2019,88.0,Thriller,Staged Killer,Staged Killer,...,"Chrishell Stause, Nicole Bilderback, Jason Dolley",English,United States,NaN,3.7,339.0,NaN,NaN,2019.0,6.0
2,2,tt10049110,7.8,96,0,2019,93.0,Documentary,Die Wiese,Die Wiese,...,Sebastian Winkler,German,Germany,NaN,7.8,95.0,NaN,NaN,2019.0,4.0


In [3]:
# Get rid of the index column in primary DF
movies_df = movies_df[['tconst', 'averageRating', 'numVotes', 'isAdult', 'startYear', 
                       'runtimeMinutes', 'genres', 'Movie', 'OMDB_Title', 'OMDB_Rated', 
                       'OMDB_Runtime', 'OMDB_Genre', 'OMDB_Director', 'OMDB_Writer', 
                       'OMDB_Actors', 'OMDB_Language', 'OMDB_Country', 'OMDB_Metascore', 
                       'OMDB_imdbRating', 'OMDB_imdbVotes', 'OMDB_BoxOffice', 'OMDB_Production', 
                       'ReleaseYear', 'ReleaseMonth']]

In [4]:
print(len(movies_df))
movies_df.head()

224162


,tconst,averageRating,numVotes,isAdult,startYear,runtimeMinutes,genres,Movie,OMDB_Title,OMDB_Rated,...,OMDB_Actors,OMDB_Language,OMDB_Country,OMDB_Metascore,OMDB_imdbRating,OMDB_imdbVotes,OMDB_BoxOffice,OMDB_Production,ReleaseYear,ReleaseMonth
0,tt10048376,4.8,30,0,2019,101.0,Drama,Ya podaryu tebe pobedu,Ya podaryu tebe pobedu,unknown,...,"Tigran Agadzhanyan, Maria Atlas, Valeriy Barinov",Russian,Russia,NaN,5.1,22.0,NaN,NaN,2019.0,4.0
1,tt10048556,3.7,340,0,2019,88.0,Thriller,Staged Killer,Staged Killer,teens,...,"Chrishell Stause, Nicole Bilderback, Jason Dolley",English,United States,NaN,3.7,339.0,NaN,NaN,2019.0,6.0
2,tt10049110,7.8,96,0,2019,93.0,Documentary,Die Wiese,Die Wiese,unknown,...,Sebastian Winkler,German,Germany,NaN,7.8,95.0,NaN,NaN,2019.0,4.0
3,tt10049238,7.1,41,0,2018,61.0,Documentary,Joan Miró: The Inner Fire,Miró contra Miró,unknown,...,"Hester Wilcox, Joan Miró, Joan Punyet Miró","Spanish, French, English","France, Spain",NaN,7.0,32.0,NaN,NaN,2018.0,10.0
4,tt10049262,5.4,176,0,2020,107.0,Comedy,James & Pat & Dave,James & Pat & Dave,unknown,...,"Ronnie Alonte, Loisa Andalio, Donny Pangilinan","Tagalog, Filipino",Philippines,NaN,5.4,174.0,NaN,NaN,2020.0,2.0


In [5]:
print(f'director df length: {len(directors_df)}')
print(f'writer df length: {len(writers_df)}')
print(f'actor df length: {len(directors_df)}')

director df length: 116495
writer df length: 163095
actor df length: 116495


# Pull in Director Average Box Office, IMDB Rating, and Combined Score

In [6]:
# Grab start time
start_time = time.time()

# Loop through every row of the movies_df
for index, row in movies_df.iterrows():
    movie_directors = row['OMDB_Director']
    # print something to indicate still working...
    if index%10000 == 0:
        print(f'working on index: {index}')
    # Only take action if the directors cell is populated with a string
    if isinstance(movie_directors, str) and movie_directors != "" :
        # Convert the comma separated list of directors into a list
        director_list = movie_directors.split(", ")
        # Set lists to store the ratings for a given row
        boxes = []
        ratings = []
        scores = []
        # Iterate through the each director in the list to get the rating for each - dropped 2 unused metrics for the sake of time
        for d in director_list:
            # Collect the values for the given director; if not found in file, set to 0
            try: 
                # d_box = directors_df.loc[directors_df['Director'] == d, 'Average_Box_Office'].iloc[0]
                d_rating = directors_df.loc[directors_df['Director'] == d, 'Average_IMDB_Rating'].iloc[0]
                # d_score = directors_df.loc[directors_df['Director'] == d, 'Success_Metric'].iloc[0]
            except IndexError:
                d_rating = 0

            # Only populate the list if a number (i.e., not NaN) to avoid creating NaN if 1 out of 6 is NaN
            # if d_box > 0: 
            #     boxes.append(d_box)
            if d_rating > 0: 
                ratings.append(d_rating)
            # if d_score > 0: 
            #     scores.append(d_score)
        # Populate the DataFrame with the average across the listed directors
        # if boxes != []:
        #     directors_box = round(np.average(boxes),2)
        #     movies_df.loc[index,'Director_Avg_Box'] = directors_box
        if ratings != []:
            directors_rating = round(np.average(ratings),2)
            movies_df.loc[index,'Director_Avg_Rating'] = directors_rating
        # if scores != []:
        #     directors_score = round(np.average(scores),2)
        #     movies_df.loc[index,'Director_Score'] = directors_score

# Print time taken
print("--- %s seconds ---" % round((time.time() - start_time),2))

movies_df.head()

working on index: 0
working on index: 10000
working on index: 20000
working on index: 30000
working on index: 40000
working on index: 50000
working on index: 60000
working on index: 70000
working on index: 80000
working on index: 90000
working on index: 100000
working on index: 110000
working on index: 120000
working on index: 130000
working on index: 140000
working on index: 150000
working on index: 160000
working on index: 170000
working on index: 180000
working on index: 190000
working on index: 200000
working on index: 210000
working on index: 220000
--- 1151.71 seconds ---


,tconst,averageRating,numVotes,isAdult,startYear,runtimeMinutes,genres,Movie,OMDB_Title,OMDB_Rated,...,OMDB_Language,OMDB_Country,OMDB_Metascore,OMDB_imdbRating,OMDB_imdbVotes,OMDB_BoxOffice,OMDB_Production,ReleaseYear,ReleaseMonth,Director_Avg_Rating
0,tt10048376,4.8,30,0,2019,101.0,Drama,Ya podaryu tebe pobedu,Ya podaryu tebe pobedu,unknown,...,Russian,Russia,NaN,5.1,22.0,NaN,NaN,2019.0,4.0,6.05
1,tt10048556,3.7,340,0,2019,88.0,Thriller,Staged Killer,Staged Killer,teens,...,English,United States,NaN,3.7,339.0,NaN,NaN,2019.0,6.0,3.89
2,tt10049110,7.8,96,0,2019,93.0,Documentary,Die Wiese,Die Wiese,unknown,...,German,Germany,NaN,7.8,95.0,NaN,NaN,2019.0,4.0,7.54
3,tt10049238,7.1,41,0,2018,61.0,Documentary,Joan Miró: The Inner Fire,Miró contra Miró,unknown,...,"Spanish, French, English","France, Spain",NaN,7.0,32.0,NaN,NaN,2018.0,10.0,7.48
4,tt10049262,5.4,176,0,2020,107.0,Comedy,James & Pat & Dave,James & Pat & Dave,unknown,...,"Tagalog, Filipino",Philippines,NaN,5.4,174.0,NaN,NaN,2020.0,2.0,5.93


# Pull in Writer Average Box Office, IMDB Rating, and Combined Score

In [7]:
# Grab start time
start_time = time.time()

# Loop through every row of the movies_df
for index, row in movies_df.iterrows():
    movie_writers = row['OMDB_Writer']
    # print something to indicate still working...
    if index%10000 == 0:
        print(f'working on index: {index}')
    # Only take action if the directors cell is populated with a string
    if isinstance(movie_writers, str) and movie_writers != "" :
        # Convert the comma separated list of directors into a list
        writer_list = movie_writers.split(", ")
        # Set lists to store the ratings for a given row
        boxes = []
        ratings = []
        scores = []
        # Iterate through the each writer in the list to get the rating for each  - dropped 2 unused metrics for the sake of time
        for w in writer_list:
            # Collect the values for the given writer; if not found in file, set to 0
            try: 
                # w_box = writers_df.loc[writers_df['Writer'] == w, 'Average_Box_Office'].iloc[0]
                w_rating = writers_df.loc[writers_df['Writer'] == w, 'Average_IMDB_Rating'].iloc[0]
                # w_score = writers_df.loc[writers_df['Writer'] == w, 'Success_Metric'].iloc[0]
            except IndexError:
                w_rating = 0

            # Only populate the list if a number (i.e., not NaN) to avoid creating NaN if 1 out of 6 is NaN
            # if w_box > 0: 
            #     boxes.append(w_box)
            if w_rating > 0: 
                ratings.append(w_rating)
            # if w_score > 0: 
            #     scores.append(w_score)
        # Populate the DataFrame with the average across the listed directors
        # if boxes != []:
        #     writers_box = round(np.average(boxes),2)
        #     movies_df.loc[index,'Writer_Avg_Box'] = writers_box
        if ratings != []:
            writers_rating = round(np.average(ratings),2)
            movies_df.loc[index,'Writer_Avg_Rating'] = writers_rating
        # if scores != []:
        #     writers_score = round(np.average(scores),2)
        #     movies_df.loc[index,'Writer_Score'] = writers_score

# Print time taken
print("--- %s seconds ---" % round((time.time() - start_time),2))

movies_df.head()

working on index: 0
working on index: 10000
working on index: 20000
working on index: 30000
working on index: 40000
working on index: 50000
working on index: 60000
working on index: 70000
working on index: 80000
working on index: 90000
working on index: 100000
working on index: 110000
working on index: 120000
working on index: 130000
working on index: 140000
working on index: 150000
working on index: 160000
working on index: 170000
working on index: 180000
working on index: 190000
working on index: 200000
working on index: 210000
working on index: 220000
--- 2032.79 seconds ---


,tconst,averageRating,numVotes,isAdult,startYear,runtimeMinutes,genres,Movie,OMDB_Title,OMDB_Rated,...,OMDB_Country,OMDB_Metascore,OMDB_imdbRating,OMDB_imdbVotes,OMDB_BoxOffice,OMDB_Production,ReleaseYear,ReleaseMonth,Director_Avg_Rating,Writer_Avg_Rating
0,tt10048376,4.8,30,0,2019,101.0,Drama,Ya podaryu tebe pobedu,Ya podaryu tebe pobedu,unknown,...,Russia,NaN,5.1,22.0,NaN,NaN,2019.0,4.0,6.05,6.05
1,tt10048556,3.7,340,0,2019,88.0,Thriller,Staged Killer,Staged Killer,teens,...,United States,NaN,3.7,339.0,NaN,NaN,2019.0,6.0,3.89,4.68
2,tt10049110,7.8,96,0,2019,93.0,Documentary,Die Wiese,Die Wiese,unknown,...,Germany,NaN,7.8,95.0,NaN,NaN,2019.0,4.0,7.54,7.74
3,tt10049238,7.1,41,0,2018,61.0,Documentary,Joan Miró: The Inner Fire,Miró contra Miró,unknown,...,"France, Spain",NaN,7.0,32.0,NaN,NaN,2018.0,10.0,7.48,NaN
4,tt10049262,5.4,176,0,2020,107.0,Comedy,James & Pat & Dave,James & Pat & Dave,unknown,...,Philippines,NaN,5.4,174.0,NaN,NaN,2020.0,2.0,5.93,5.29


# Pull in Actor/Actress Average Box Office, IMDB Rating, and Combined Score

In [11]:
# Grab start time
start_time = time.time()

# Loop through every row of the movies_df
for index, row in movies_df.iterrows():
    movie_actors = row['OMDB_Actors']
    # print something to indicate still working...
    if index%10000 == 0:
        print(f'working on index: {index}')
    # Only take action if the directors cell is populated with a string
    if isinstance(movie_actors, str) and movie_actors != "" :
        # Convert the comma separated list of directors into a list
        actor_list = movie_actors.split(", ")
        # Set lists to store the ratings for a given row
        boxes = []
        ratings = []
        scores = []
        # Iterate through the each actor in the list to get desired values - dropped 2 unused metrics for the sake of time
        for a in actor_list:
            # Collect the values for the given actor
            try:
                # a_box = actors_df.loc[actors_df['Actor'] == a, 'Average_Box_Office'].iloc[0]
                a_rating = actors_df.loc[actors_df['Actor'] == a, 'Average_IMDB_Rating'].iloc[0]
                # a_score = actors_df.loc[actors_df['Actor'] == a, 'Success_Metric'].iloc[0]
            except IndexError:
                a_rating = 0
                
            # Only populate the list if a number (i.e., not NaN) to avoid creating NaN if 1 out of 6 is NaN
            # if a_box > 0: 
            #     boxes.append(a_box)
            if a_rating > 0: 
                ratings.append(a_rating)
            # if a_score > 0: 
            #     scores.append(a_score)
        # Populate the DataFrame with the average across the listed directors
        # if boxes != []:
        #     actors_box = round(np.average(boxes),2)
        #     movies_df.loc[index,'Actor_Avg_Box'] = actors_box
        if ratings != []:
            actors_rating = round(np.average(ratings),2)
            movies_df.loc[index,'Actor_Avg_Rating'] = actors_rating
        # if scores != []:
        #     actors_score = round(np.average(scores),2)
        #     movies_df.loc[index,'Actor_Score'] = actors_score

# Print time taken
print("--- %s seconds ---" % round((time.time() - start_time),2))

movies_df.head()

working on index: 0
working on index: 10000
working on index: 20000
working on index: 30000
working on index: 40000
working on index: 50000
working on index: 60000
working on index: 70000
working on index: 80000
working on index: 90000
working on index: 100000
working on index: 110000
working on index: 120000
working on index: 130000
working on index: 140000
working on index: 150000
working on index: 160000
working on index: 170000
working on index: 180000
working on index: 190000
working on index: 200000
working on index: 210000
working on index: 220000
--- 6876.83 seconds ---


,tconst,averageRating,numVotes,isAdult,startYear,runtimeMinutes,genres,Movie,OMDB_Title,OMDB_Rated,...,OMDB_Metascore,OMDB_imdbRating,OMDB_imdbVotes,OMDB_BoxOffice,OMDB_Production,ReleaseYear,ReleaseMonth,Director_Avg_Rating,Writer_Avg_Rating,Actor_Avg_Rating
0,tt10048376,4.8,30,0,2019,101.0,Drama,Ya podaryu tebe pobedu,Ya podaryu tebe pobedu,unknown,...,NaN,5.1,22.0,NaN,NaN,2019.0,4.0,6.05,6.05,5.03
1,tt10048556,3.7,340,0,2019,88.0,Thriller,Staged Killer,Staged Killer,teens,...,NaN,3.7,339.0,NaN,NaN,2019.0,6.0,3.89,4.68,4.94
2,tt10049110,7.8,96,0,2019,93.0,Documentary,Die Wiese,Die Wiese,unknown,...,NaN,7.8,95.0,NaN,NaN,2019.0,4.0,7.54,7.74,7.80
3,tt10049238,7.1,41,0,2018,61.0,Documentary,Joan Miró: The Inner Fire,Miró contra Miró,unknown,...,NaN,7.0,32.0,NaN,NaN,2018.0,10.0,7.48,NaN,7.17
4,tt10049262,5.4,176,0,2020,107.0,Comedy,James & Pat & Dave,James & Pat & Dave,unknown,...,NaN,5.4,174.0,NaN,NaN,2020.0,2.0,5.93,5.29,5.90


In [12]:
movies_df.to_csv("../Resources/IMDB_OMDB_FINAL.csv")